<a href="https://colab.research.google.com/github/zaenal-abidin-syah/machine-translation-transformer/blob/main/MT_id_to_en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Model

In [ ]:
class InputEmbeddings(nn.Module):
  def __init__(self, d_model : int, vocab_size):
    super().__init__()
    self.d_model = d_model
    self.vocab_size = vocab_size
    self.embedding = nn.Embedding(self.vocab_size, self.d_model)
  def forward(self, x):
    return self.embedding(x) * math.sqrt(self.d_model)

In [ ]:
class PositionalEmbeddings(nn.Module):
  def __init__(self, d_model: int, seq_len: int, dropout: float):
    super().__init__()
    self.d_model = d_model
    self.seq_len = seq_len
    self.dropout = nn.Dropout(dropout)

    # create matrixof shape (seq_len, d_model)
    pe = torch.zeros(self.seq_len, self.d_model)
    # create matrixof shape (seq_len, 1)
    position = torch.arange(0, self.seq_len, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, self.d_model, 2).float() * (-math.log(10000.0)/ d_model))

    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)

    pe = pe.unsqueeze(0) # (1, seq_len, d_model)
    self.register_buffer('pe', pe)
  def forward(self, x):
    x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False)
    return self.dropout(x)

In [ ]:
class LayerNormalization(nn.Module):
  def __init__(self, eps: float = 10 ** -6):
    super().__init__()
    self.eps = eps
    self.alpha = nn.Parameter(torch.ones(1)) # Multiplied
    self.bias = nn.Parameter(torch.zeros(1)) # added
  def forward(self, x):
    mean = x.mean(dim=-1, keepdim=True)
    std = x.std(dim=-1, keepdim=True)
    return self.alpha * (x - mean) / (std + self.eps) + self.bias

In [ ]:
class FeedForwardBlock(nn.Module):
  def __init__(self, d_model: int, d_ff: int, dropout: float):
    super().__init__()
    self.linear_1 = nn.Linear(d_model, d_ff)
    self.dropout = nn.Dropout(dropout)
    self.linear_2 = nn.Linear(d_ff, d_model)
  def forward(self, x):
    x = torch.relu(self.linear_1(x))
    x = self.dropout(x)
    x = self.linear_2(x)
    return x

In [ ]:
class MultiHeadAttentionBlock(nn.Module):
  def __init__(self, d_model: int, h: int, dropout: float):
    super().__init__()
    self.d_model = d_model
    self.h = h
    assert self.d_model % h == 0, "d_model is not divisible by h"

    self.d_k = self.d_model // self.h
    self.w_q = nn.Linear(self.d_model, self.d_model)
    self.w_k = nn.Linear(self.d_model, self.d_model)
    self.w_v = nn.Linear(self.d_model, self.d_model)

    self.w_o = nn.Linear(self.d_model, self.d_model)
    self.dropout = nn.Dropout(dropout)
  @staticmethod
  def attention(query, key, value, mask, dropout: nn.Dropout):
    d_k = query.shape[-1]

    # (batch, h, seq_len, d_k)
    attention_score = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
      attention_score.masked_fill_(mask==0, -1e9)
    attention_score = attention_score.softmax(dim=-1)
    if dropout is not None:
      attention_score = dropout(attention_score)
    return (attention_score @ value), attention_score
  def forward(self, q, k, v, mask):
    query = self.w_q(q)
    key = self.w_k(k)
    value = self.w_v(v)

    query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1,2)
    key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1,2)
    value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1,2)

    x, self.attention_score = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)

    x = x.transpose(1,2).contiguous().view(x.shape[0], -1, self.h * self.d_k)
    return self.w_o(x)

In [ ]:
class ResidualConnection(nn.Module):
  def __init__(self, dropout: float):
    super().__init__()
    self.dropout = nn.Dropout(dropout)
    self.norm = LayerNormalization()
  def forward(self, x, sublayer):
    return x + self.dropout(sublayer(self.norm(x)))

In [ ]:
class EncoderBlock(nn.Module):
  def __init__(self, self_attention_block: MultiHeadAttentionBlock, feedforward_block: FeedForwardBlock, dropout: float):
    super().__init__()
    self.self_attention_block = self_attention_block
    self.feedforward_block = feedforward_block
    self.residual_connection = nn.ModuleList([ResidualConnection(dropout) for _ in range(2)])
  def forward(self, x, src_mask):
    x = self.residual_connection[0](x, lambda x : self.self_attention_block(x,x,x, src_mask))
    x = self.residual_connection[1](x, self.feedforward_block)
    return x

In [ ]:
class Encoder(nn.Module):
  def __init__(self, layers: nn.ModuleList):
    super().__init__()
    self.layers = layers
    self.norm = LayerNormalization()
  def forward(self, x, mask):
    for layer in self.layers:
      x = layer(x, mask)
    return self.norm(x)

In [ ]:
class DecoderBlock(nn.Module):
  def __init__(self, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feedforward_block: FeedForwardBlock, dropout: float):
    super().__init__()
    self.self_attention_block = self_attention_block
    self.cross_attention_block = cross_attention_block
    self.feedforward_block = feedforward_block
    self.residual_connection = nn.ModuleList([ResidualConnection(dropout) for _ in range(3)])
  def forward(self, x, encoder_output, src_mask, tgt_mask):
    x = self.residual_connection[0](x, lambda x : self.self_attention_block(x, x, x, tgt_mask))
    x = self.residual_connection[1](x, lambda x : self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
    x = self.residual_connection[2](x, self.feedforward_block)
    return x

In [ ]:
class Decoder(nn.Module):
  def __init__(self, layers: nn.ModuleList):
    super().__init__()
    self.layers = layers
    self.norm = LayerNormalization()
  def forward(self, x, encoder_output, src_mask, tgt_mask):
    for layer in self.layers:
      x = layer(x, encoder_output, src_mask, tgt_mask)
    return self.norm(x)

In [ ]:
class ProjectionLayer(nn.Module):
  def __init__(self, d_model: int, vocab_size: int):
    super().__init__()
    self.projection = nn.Linear(d_model, vocab_size)
  def forward(self, x):
    return torch.log_softmax(self.projection(x), dim=-1)

In [ ]:
class Transformer(nn.Module):
  def __init__(self,
    encoder: Encoder,
    decoder: Decoder,
    src_embedding: InputEmbeddings,
    tgt_embedding: InputEmbeddings,
    src_position: PositionalEmbeddings,
    tgt_position: PositionalEmbeddings,
    projection_layer: ProjectionLayer):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.src_embedding = src_embedding
    self.tgt_embedding = tgt_embedding
    self.src_position = src_position
    self.tgt_position = tgt_position
    self.projection_layer = projection_layer

  def encode(self, src, src_mask):
    src = self.src_embedding(src)
    src = self.src_position(src)
    return self.encoder(src, src_mask)

  def decode(self, encoder_output, src_mask, tgt, tgt_mask):
    tgt = self.tgt_embedding(tgt)
    tgt = self.tgt_position(tgt)
    return self.decoder(tgt, encoder_output, src_mask, tgt_mask)

In [ ]:
def build_transformer(
  src_vocab_size: int,
  tgt_vocab_size: int,
  src_seq_len: int,
  tgt_seq_len: int,
  d_model: int = 128,
  N: int = 2,
  h: int=4,
  dropout: float=0.1,
  d_ff: int = 2048
)-> Transformer:
  # create embeddings layer
  src_embed = InputEmbeddings(d_model, src_vocab_size)
  tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

  # create positional embedding
  src_pos = PositionalEmbeddings(d_model, src_seq_len, dropout)
  tgt_pos = PositionalEmbeddings(d_model, tgt_seq_len, dropout)

  # create encoder blocks
  encoder_blocks = []
  for _ in range(N):
    encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
    feedforward_block = FeedForwardBlock(d_model, d_ff, dropout)
    encoder_block = EncoderBlock(encoder_self_attention_block, feedforward_block, dropout)
    encoder_blocks.append(encoder_block)

  # create decoder blocks
  decoder_blocks = []
  for _ in range(N):
    decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
    decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
    feedforward_block = FeedForwardBlock(d_model, d_ff, dropout)
    decoder_block = DecoderBlock(decoder_self_attention_block, decoder_cross_attention_block, feedforward_block, dropout)
    decoder_blocks.append(decoder_block)

  # create encoder and decoder
  encoder = Encoder(nn.ModuleList(encoder_blocks))
  decoder = Decoder(nn.ModuleList(decoder_blocks))

  # projection
  projection_layer = ProjectionLayer(d_model, tgt_vocab_size)

  # create transformer
  transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)

  for p in transformer.parameters():
    if p.dim() > 1:
      nn.init.xavier_uniform_(p)
  return transformer

# Config

In [ ]:
def get_config(model_name, preload=None, folder='.', epoch=10):
  return {
    "model_name": model_name,
    "batch_size": 32,
    "num_epochs": epoch,
    "lr": 10**-4,
    "seq_len": 80,
    "d_model": 64,
    "lang_src": "indonesian",
    "lang_tgt": "english",
    "model_folder": f"{folder}/weights_{model_name}",
    "model_basename": "tmodel_",
    "preload": preload,
    "tokenizer_file": folder + "/tokenizer_{0}.json",
    "experiment_name": f"{folder}/runs_{model_name}/tmodel"
  }

def get_weights_file_path(config, epoch: str):
  model_folder = config["model_folder"]
  model_basename = config["model_basename"]
  model_filename = f"{model_basename}{epoch}.pt"
  return str(Path(".") / model_folder / model_filename)

In [ ]:
FOLDER_PATH = '/content/drive/MyDrive/Machine_translation'

In [ ]:
preload = [None, None, None]
configs =[ get_config(f"model_{i}", folder=FOLDER_PATH, preload=preload[i-1]) for i in range(1, 4)]
configs

[{'model_name': 'model_1',
  'batch_size': 32,
  'num_epochs': 10,
  'lr': 0.0001,
  'seq_len': 80,
  'd_model': 64,
  'lang_src': 'indonesian',
  'lang_tgt': 'english',
  'model_folder': '/content/drive/MyDrive/Machine_translation/weights_model_1',
  'model_basename': 'tmodel_',
  'preload': None,
  'tokenizer_file': '/content/drive/MyDrive/Machine_translation/tokenizer_{0}.json',
  'experiment_name': '/content/drive/MyDrive/Machine_translation/runs_model_1/tmodel'},
 {'model_name': 'model_2',
  'batch_size': 32,
  'num_epochs': 10,
  'lr': 0.0001,
  'seq_len': 80,
  'd_model': 64,
  'lang_src': 'indonesian',
  'lang_tgt': 'english',
  'model_folder': '/content/drive/MyDrive/Machine_translation/weights_model_2',
  'model_basename': 'tmodel_',
  'preload': None,
  'tokenizer_file': '/content/drive/MyDrive/Machine_translation/tokenizer_{0}.json',
  'experiment_name': '/content/drive/MyDrive/Machine_translation/runs_model_2/tmodel'},
 {'model_name': 'model_3',
  'batch_size': 32,
  'num_

# Dataset

## Masking

In [ ]:
def causal_mask(size):
  mask = torch.triu(torch.ones(1, size, size), diagonal=1).type(torch.int)
  return mask == 0

## Custom Dataset

In [ ]:
class BilingualDataset(Dataset):
  def __init__(self, ds, tokenizer_src, tokenizer_tgt, src_lang, tgt_lang, seq_len):
    self.ds = ds
    self.tokenizer_src = tokenizer_src
    self.tokenizer_tgt = tokenizer_tgt
    self.src_lang = src_lang
    self.tgt_lang = tgt_lang
    self.seq_len = seq_len

    # print("tokenizer type:", type(tokenizer_src))
    # for t in ["[SOS]","[EOS]","[PAD]","[UNK]"]:
    #   val = tokenizer_src.token_to_id(t)
    #   print(f"{t!r} ->", val, " type:", type(val))
    # # kalau ada method get_vocab:
    # if hasattr(tokenizer_src, "get_vocab"):
    #   print("vocab sample:", list(list(tokenizer_src.get_vocab().items())[:20]))
    # elif hasattr(tokenizer_src, "get_vocab_size"):
    #   print("vocab size:", tokenizer_src.get_vocab_size())
    # else:
    #   print("No get_vocab method available on tokenizer object.")

    # self.sos_token = torch.Tensor([tokenizer_src.token_to_id(["[SOS]"])], dtype=torch.int64)
    # self.eos_token = torch.Tensor([tokenizer_src.token_to_id(["[EOS]"])], dtype=torch.int64)
    # self.pad_token = torch.Tensor([tokenizer_src.token_to_id(["[PAD]"])], dtype=torch.int64)

    self.sos_token = torch.tensor([tokenizer_src.token_to_id("[SOS]")], dtype=torch.int64)
    self.eos_token = torch.tensor([tokenizer_src.token_to_id("[EOS]")], dtype=torch.int64)
    self.pad_token = torch.tensor([tokenizer_src.token_to_id("[PAD]")], dtype=torch.int64)

    print("apakah ada masalah di sos eos dan pad token?")

  def __len__(self):
    return len(self.ds)
  def __getitem__(self, index):
    src_target_pair = self.ds[index]
    src_text = src_target_pair[self.src_lang]
    tgt_text = src_target_pair[self.tgt_lang]

    enc_input_tokens = self.tokenizer_src.encode(src_text).ids
    dec_input_tokens = self.tokenizer_tgt.encode(tgt_text).ids

    enc_num_padding_tokens = self.seq_len - len(enc_input_tokens) - 2
    dec_num_padding_tokens = self.seq_len - len(dec_input_tokens) - 1

    if enc_num_padding_tokens < 0 or dec_num_padding_tokens < 0:
      raise ValueError("Sentence is to long")

    # Add sos and eos to source text
    encoder_input = torch.cat(
      [
        self.sos_token,
        torch.tensor(enc_input_tokens, dtype=torch.int64),
        self.eos_token,
        torch.tensor([self.pad_token] * enc_num_padding_tokens, dtype=torch.int64)
      ]
    )

    # add sos to decoder input
    decoder_input = torch.cat(
      [
        self.sos_token,
        torch.tensor(dec_input_tokens, dtype=torch.int64),
        torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64)
      ]
    )

    # add eos  to the label (what we expect as output from decoder)
    label = torch.cat(
      [
        torch.tensor(dec_input_tokens, dtype=torch.int64),
        self.eos_token,
        torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64)
      ]
    )

    assert encoder_input.size(0) == self.seq_len
    assert decoder_input.size(0) == self.seq_len
    assert label.size(0) == self.seq_len
    return {
      "encoder_input": encoder_input,
      "decoder_input": decoder_input,
      "encoder_mask": (encoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int(),
      "decoder_mask": (decoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int() & causal_mask(decoder_input.size(0)),
      "label": label,
      "src_text": src_text,
      "tgt_text": tgt_text
    }


# Train

In [ ]:
# !pip install datasets tokenizers
# !pip install tensorboard

In [ ]:
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

from pathlib import Path
from tqdm import tqdm
import warnings
from torch.utils.tensorboard import SummaryWriter
# from config import get_config, get_weights_file_path
# from dataset import BilingualDataset, causal_mask

## Greedy decode

In [ ]:
# def greedy_decode(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device):
#   sos_idx = tokenizer_tgt.token_to_id("[SOS]")
#   eos_idx = tokenizer_tgt.token_to_id("[EOS]")

#   encoder_output = model.encode(source, source_mask)

#   decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(source).to(device)
#   while True:
#     if decoder_input.size(1) == max_len:
#       break
#     decoder_mask = causal_mask(decoder_input.size(1)).type_as(source_mask).to(device)

#     out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

#     prob = model.projection_layer(out[:, -1])

#     _, next_word = torch.max(prob, dim = 1)
#     decoder_input = torch.cat([decoder_input, torch.empty(1,1).type_as(source).fill_(next_word.item()).to(device)], dim=1)
#     if next_word == eos_idx:
#       break
#   return decoder_input.squeeze(0)

In [ ]:
import torch

def greedy_decode_batch(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device):
    sos_idx = tokenizer_tgt.token_to_id("[SOS]")
    eos_idx = tokenizer_tgt.token_to_id("[EOS]")

    batch_size = source.size(0)
    encoder_output = model.encode(source, source_mask)

    # start token untuk seluruh batch
    decoder_input = torch.full((batch_size, 1), sos_idx, dtype=source.dtype, device=device)

    # track which sequences sudah selesai (muncul EOS)
    finished = torch.zeros(batch_size, dtype=torch.bool, device=device)

    for _step in range(max_len - 1):  # already punya SOS, jadi max_len-1 langkah tambah
        tgt_len = decoder_input.size(1)
        decoder_mask = causal_mask(tgt_len).type_as(source_mask).to(device)

        out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)
        # out shape asumsi: (batch_size, tgt_len, hidden)
        logits = model.projection_layer(out[:, -1, :])  # ambil logits token terakhir -> (batch_size, vocab_size)

        # pilih token dengan probabilitas tertinggi per batch item (greedy)
        next_word = torch.argmax(logits, dim=-1)  # (batch_size,)

        # jika sudah finished sebelumnya, tetap isi dengan EOS supaya tidak berubah
        next_word = torch.where(finished, torch.full_like(next_word, eos_idx), next_word)

        # append ke decoder_input
        decoder_input = torch.cat([decoder_input, next_word.unsqueeze(1)], dim=1)  # (batch_size, tgt_len+1)

        # update finished flags
        finished = finished | (next_word == eos_idx)

        # kalau semua selesai, stop lebih awal
        if finished.all():
            break

    return decoder_input  # (batch_size, decoded_len)

## validation

In [ ]:
!pip install -q evaluate contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.8 MB/s eta 0:00:00


In [ ]:
# def run_validation(model, validation_ds, tokenizer_src, tokenizer_tgt, max_len, device, print_msg, global_state, writer, num_examples=5):
#   model.eval()
#   count = 0

#   preds = []
#   refs = []
#   # expected = []
#   # predicted = []

#   console_width = 80

#   with torch.no_grad():
#     for batch in validation_ds:
#       count += 1
#       encoder_input = batch["encoder_input"].to(device)
#       encoder_mask = batch["encoder_mask"].to(device)
#       assert encoder_input.size(0) == 1, "Batch size must be 1 for validation"

#       model_out = greedy_decode(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, max_len, device)

#       source_text = batch["src_text"][0]
#       target_text = batch["tgt_text"][0]
#       model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy())

#       # source_texts.append(source_text)
#       # expected.append(target_text)
#       # predicted.append(model_out_text)
#       preds.append(model_out_text)
#       refs.append(target_text)

#       print_msg("-" * console_width)
#       print_msg(f"Source: {source_text}")
#       print_msg(f"Target: {target_text}")
#       print_msg(f"Predicted: {model_out_text}")
#       if count == num_examples:
#         break

#   print("hitung blue")
#   if writer:
#     bleu_metric = load("bleu")
#     result = bleu_metric.compute(predictions=preds, references=refs)
#     print(result)
#   #   # torchmetrics CharErrorRate, BLUE, WordErrorRate, etc can be used here
#   #   pass

In [ ]:
import torch
from evaluate import load  # make sure `evaluate` package tersedia

def run_validation(model, validation_ds, tokenizer_src, tokenizer_tgt, max_len, device, print_msg, global_state, writer, num_examples=5):
    model.eval()
    preds = []
    refs = []

    console_width = 80

    sos_idx = tokenizer_tgt.token_to_id("[SOS]")
    eos_idx = tokenizer_tgt.token_to_id("[EOS]")

    with torch.no_grad():
        for batch in validation_ds:
            encoder_input = batch["encoder_input"].to(device)
            encoder_mask = batch["encoder_mask"].to(device)

            # decode batch-wise (mengembalikan tensor (batch_size, decoded_len))
            model_out = greedy_decode_batch(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, max_len, device)
            # model_out: torch.LongTensor, shape (batch_size, seq_len)

            # convert to python lists of token ids and trim SOS/EOS
            seqs = model_out.detach().cpu().numpy().tolist()
            cleaned = []
            for seq in seqs:
                # drop leading SOS if present
                if len(seq) > 0 and seq[0] == sos_idx:
                    seq = seq[1:]
                # cut at EOS (exclude EOS from decoded ids)
                if eos_idx in seq:
                    seq = seq[:seq.index(eos_idx)]
                cleaned.append(seq)

            # decode to text: prefer batch_decode if available
            if hasattr(tokenizer_tgt, "batch_decode"):
                # some tokenizers expect list[int] per example
                model_out_texts = tokenizer_tgt.batch_decode(cleaned, skip_special_tokens=True)
            else:
                model_out_texts = [tokenizer_tgt.decode(ids, skip_special_tokens=True) for ids in cleaned]

            batch_size = encoder_input.size(0)
            for i in range(batch_size):
                src_text = batch["src_text"][i]
                tgt_text = batch["tgt_text"][i]
                pred_text = model_out_texts[i]

                preds.append(pred_text)
                refs.append(tgt_text)

                # only print up to num_examples total
                if len(preds) <= num_examples:
                    print_msg("-" * console_width)
                    print_msg(f"Source: {src_text}")
                    print_msg(f"Target: {tgt_text}")
                    print_msg(f"Predicted: {pred_text}")

            if len(preds) >= num_examples:
                break

    # compute BLEU (or other metric)
    try:
        bleu_metric = load("bleu")
        # note: some evaluate BLEU implementations expect references as list[list[str]]
        # if that API is required in your env, wrap refs: [[r] for r in refs]
        result = bleu_metric.compute(predictions=preds, references=refs)
        print("BLEU:", result)
        # if you have a writer (tensorboard) log it
        if writer:
            # adjust the key/name according to your writer usage
            if isinstance(result, dict):
                # try to log a numeric value if present
                if "bleu" in result:
                    writer.add_scalar("val/bleu", result["bleu"], global_state.get("step", 0) if global_state else 0)
            else:
                # result might be a single numeric
                writer.add_scalar("val/bleu", float(result), global_state.get("step", 0) if global_state else 0)
    except Exception as e:
        print(f"Could not compute BLEU metric: {e}")

    return {"preds": preds, "refs": refs}


# Get dataset

In [ ]:
from datasets import Dataset as HFDataset

In [ ]:
import re
import string
import html

## Preprocessing

In [ ]:
import contractions

In [ ]:
EMOJI_PATTERN = re.compile(
    "["
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F1E0-\U0001F1FF"  # flags
    "\U00002700-\U000027BF"
    "\U000024C2-\U0001F251"
    "]+",
    flags=re.UNICODE
)

RE_EMAIL = re.compile(r'\b[\w\.-]+@[\w\.-]+\.\w{2,}\b', flags=re.UNICODE)
RE_URL = re.compile(
    r'((http|https):\/\/[^\s]+)|(www\.[^\s]+)|(\b[\w-]+\.(com|net|org|edu|id|gov|io|co|biz|info|me)(\/[^\s]*)?)',
    flags=re.IGNORECASE
)
RE_HANDLE = re.compile(r'@\w+')
RE_HTML_TAG = re.compile(r'<[^>]+>')
RE_HTML_ENTITY = re.compile(r'&[a-zA-Z]+;')
RE_MUSIC_NOTES = re.compile(r'[\u266B\u266A\u2669\u266C♫♪]+')
# RE_WORD_BRACKET = re.compile(r'[\[\{\(]*[.][\]\}\)]')
RE_WORD_BRACKET = re.compile(r'[\[\{\(][^\]\}\)]*[\]\}\)]', flags=re.UNICODE)


def preprocess_text(
    text,
    lowercase=True,
    remove_emails=True,
    remove_urls=True,
    remove_word_bracket=True,
    remove_music_notes=True,
    remove_html=True,
    remove_handles=True,
    contraction=False,
    remove_emoji=True,
    remove_ascii_punct=True
):
    """
    Bersihkan 1 string text. Semua opsional bisa dimatikan sesuai kebutuhan.
    """
    if text is None:
        return text
    # 1) unescape HTML entities seperti &lt;i&gt; -> <i>
    text = html.unescape(text)

    # 2) lowercase
    if lowercase:
        text = text.lower()
        # print("lowercase : ",text)

    if contraction:
      text = contractions.fix(text)

    # 3) hapus tag HTML
    if remove_html:
        text = RE_HTML_TAG.sub(' ', text)
        # print("remove html : ",text)


    # 4) hapus email/url/handle
    if remove_emails:
        text = RE_EMAIL.sub(' ', text)
        # print("remove email : ",text)
    if remove_urls:
        text = RE_URL.sub(' ', text)
        # print("remove url : ",text)
    if remove_word_bracket:
        text = RE_WORD_BRACKET.sub(' ', text)
        # print("remove word bracket : ",text)
    if remove_handles:
        text = RE_HANDLE.sub(' ', text)
        # print("remove handle : ",text)

    # 5) hapus simbol musik
    if remove_music_notes:
        text = RE_MUSIC_NOTES.sub(' ', text)
        # print("remove music notes : ",text)

    # 6) hapus emoji
    if remove_emoji:
        text = EMOJI_PATTERN.sub('', text)
        # print("remove emoji : ",text)

    # 11) hapus leftover HTML entities seperti &lt;
    text = RE_HTML_ENTITY.sub(' ', text)
    # print("remove html entity : ",text)
    # 12) hapus punctuation ascii (jika diinginkan)
    if remove_ascii_punct:
        text = re.sub(r'--', '', text)
        remove = ".,\'\"-$?!"
        # bikin versi custom string.punctuation
        custom_punct = string.punctuation.translate(str.maketrans('', '', remove))
        text = text.translate(str.maketrans('', '', custom_punct))
        # print("remove ascii punct : ",text)

    # 13) collapse multiple spaces dan trim
    text = ' '.join(text.split())

    return text

## Get data from paralel text

In [ ]:
def read_paralel_text(file_lang_src, file_lang_tgt, lang_src, lang_tgt, max_len):
  with open(file_lang_src, "r", encoding="utf-8") as f_src, open(file_lang_tgt, "r", encoding="utf-8") as f_tgt:
    data = 0
    pairs = []
    id = 0
    max_len = max_len - 20
    max_length = 0

    for src_lines, tgt_lines in zip(f_src, f_tgt):
      data += 1
      src_lines, tgt_lines = src_lines.strip(), tgt_lines.strip()
      # Preprocess
      src_lines, tgt_lines = preprocess_text(src_lines), preprocess_text(tgt_lines, contraction=True)

      max_length = max(max_length, len(src_lines.split()), len(tgt_lines.split()))
      if len(src_lines.split()) > max_len or len(tgt_lines.split()) > max_len:
        continue
      elif len(src_lines) == 0 or len(tgt_lines) == 0:
        continue
      else:
        pairs.append({
            lang_src : src_lines,
            lang_tgt : tgt_lines,
            "id" : id
          })
        id += 1
    print("Jumlah data : ", data)
    print("max word : ", max_length)
    print("Jumlah data yang dihapus : ", data - id)
    # print("max word : ", max_word)
  ds = HFDataset.from_list(pairs)
  return ds

In [ ]:
ID = f"{FOLDER_PATH}/TED2020.en-id.id"
EN = f"{FOLDER_PATH}/TED2020.en-id.en"

dataset = read_paralel_text(ID, EN, "indonesian", "english", 64)

Jumlah data :  165059
max word :  521
Jumlah data yang dihapus :  7704


### save and load dataset

In [ ]:
dataset.save_to_disk(f"{FOLDER_PATH}/data/dataset")

Saving the dataset (0/1 shards):   0%|          | 0/157355 [00:00<?, ? examples/s]

In [ ]:
dataset = HFDataset.load_from_disk(f"{FOLDER_PATH}/data/dataset")

# Build Tokenizer

In [ ]:
def get_all_sentences(ds, lang):
  for item in ds:
    yield item[lang]

def get_or_build_tokenizer(config, ds, lang):
  tokenizer_path = Path(config["tokenizer_file"].format(lang)).expanduser().resolve()
  print("path tokenizer : ", tokenizer_path)
  if not Path.exists(tokenizer_path):
    print("tokenizer file tidak ada?")
    tokenizer = Tokenizer(WordLevel(unk_token='[UNK]'))
    tokenizer.pre_tokenizer = Whitespace()
    trainer = WordLevelTrainer(special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency=2)
    tokenizer.train_from_iterator(get_all_sentences(ds, lang), trainer=trainer)
    tokenizer.save(str(tokenizer_path))
  else:
    print("tokenizer file ada")
    tokenizer = Tokenizer.from_file(str(tokenizer_path))
  return tokenizer

#### build or get tokenizer

In [ ]:
config = configs[0]
tokenizer_src = get_or_build_tokenizer(config, dataset, config['lang_src'])
tokenizer_tgt = get_or_build_tokenizer(config, dataset, config['lang_tgt'])

path tokenizer :  /content/drive/MyDrive/Machine_translation/tokenizer_indonesian.json
tokenizer file ada
path tokenizer :  /content/drive/MyDrive/Machine_translation/tokenizer_english.json
tokenizer file ada


# Split data

In [ ]:
split_ds = dataset.train_test_split(test_size=0.2, shuffle=True)
train_ds_raw = split_ds["train"]
val_ds_raw = split_ds["test"]

## save and load split dataset

In [ ]:
train_ds_raw.save_to_disk(f"{FOLDER_PATH}/data/train-dataset")
val_ds_raw.save_to_disk(f"{FOLDER_PATH}/data/val-dataset")

Saving the dataset (0/1 shards):   0%|          | 0/125884 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/31471 [00:00<?, ? examples/s]

In [ ]:
train_ds_raw = HFDataset.load_from_disk(f"{FOLDER_PATH}/data/train-dataset")
val_ds_raw = HFDataset.load_from_disk(f"{FOLDER_PATH}/data/val-dataset")

# Bagging(Bootsrap Aggregating )

In [ ]:
def bagging(dataset, model_name, folder='.'):
  data_size = len(dataset)
  indices = torch.randint(0, data_size, (data_size,))
  indices_path = Path(f"{folder}/data/indices_{model_name}.idx").expanduser().resolve()
  # save indices to file json
  if not Path.exists(indices_path):
    with open(indices_path, "w") as f:
      for idx in indices:
        f.write(f"{idx}\n")
  subset_path = Path(f"{folder}/data/subset_{model_name}").expanduser().resolve()
  # print("path subset : ", subset_path)
  if Path.exists(subset_path):
    print(f"Loading subset for {model_name} from disk.")
    subset = HFDataset.load_from_disk(str(subset_path))
  else:
    print(f"Creating and saving subset for {model_name}.")
    with open(indices_path, "r") as f:
      indices = f.read().split()
      indices = [int(i) for i in indices]
    subset = dataset.select(indices)
    subset.save_to_disk(str(subset_path))
  return subset

## build or load subset bagging dataset models

In [ ]:
for config in configs:
  subset = bagging(train_ds_raw, config['model_name'], folder=FOLDER_PATH)

Loading subset for model_1 from disk.
Loading subset for model_2 from disk.
Loading subset for model_3 from disk.


# Build Dataloader

In [ ]:
def get_ds(config, subset, tokenizer_src, tokenizer_tgt, test=False):
  dataset = BilingualDataset(subset, tokenizer_src, tokenizer_tgt, config["lang_src"], config["lang_tgt"], config["seq_len"])

  max_len_src = 0
  max_len_tgt = 0

  for item in subset:
    src_ids = tokenizer_src.encode(item[config["lang_src"]]).ids
    tgt_ids = tokenizer_tgt.encode(item[config["lang_tgt"]]).ids

    max_len_src = max(max_len_src, len(src_ids))
    max_len_tgt = max(max_len_tgt, len(tgt_ids))

  print("max length of source sentence", max_len_src)
  print("max length of target sentence", max_len_tgt)
  batch_size = 1 if test else config["batch_size"]
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
  # val_dataloader = DataLoader(val_ds, batch_size=1)
  return dataloader


# Train Model

In [ ]:
def get_model(config, vocab_src_len, vocab_tgt_len):
  model = build_transformer(vocab_src_len, vocab_tgt_len, config["seq_len"], config["seq_len"])
  return model

In [ ]:
def train_model(config, train_subset, val_dataloader, tokenizer_src, tokenizer_tgt):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print("using device : ", device)

  Path(config["model_folder"]).mkdir(parents=True, exist_ok=True)
  train_dataloader = get_ds(config, train_subset, tokenizer_src, tokenizer_tgt)
  model = get_model(config=config, vocab_src_len=tokenizer_src.get_vocab_size(), vocab_tgt_len=tokenizer_tgt.get_vocab_size()).to(device=device)

  # tensorboard
  writer = SummaryWriter(config["experiment_name"])

  optimizer = optim.Adam(model.parameters(), lr=config["lr"], eps=1e-9)

  initial_epoch = 0
  global_step = 0
  if config["preload"]:
    model_filename = get_weights_file_path(config, config["preload"])
    print(f"Preloading model {model_filename}")
    state = torch.load(model_filename)
    model.load_state_dict(state["model_state_dict"])
    initial_epoch = state["epoch"] + 1
    optimizer.load_state_dict(state["optimizer_state_dict"])
    global_step = state["global_step"]

  loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer_tgt.token_to_id('[PAD]'), label_smoothing=0.1).to(device=device)
  for epoch in range(initial_epoch, config["num_epochs"]):
    batch_iterator = tqdm(train_dataloader, desc=f"Processing  epoch {epoch:02d}")
    for batch in batch_iterator:
      model.train()
      encoder_input = batch["encoder_input"].to(device)
      decoder_input = batch["decoder_input"].to(device)
      encoder_mask = batch["encoder_mask"].to(device)
      decoder_mask = batch["decoder_mask"].to(device)

      encoder_output = model.encode(encoder_input, encoder_mask)
      decoder_output = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask) # type: ignore
      proj_output = model.projection_layer(decoder_output)

      label = batch["label"].to(device)

      loss = loss_fn(proj_output.view(-1, tokenizer_tgt.get_vocab_size()), label.view(-1))

      batch_iterator.set_postfix({"Loss" : f"{loss.item():6.3f}"})

      # log loss
      writer.add_scalar("train loss", loss.item(), global_step)
      writer.flush()

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config["seq_len"], device, lambda msg: batch_iterator.write(msg), global_step, writer)

      global_step += 1

    run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config["seq_len"], device, lambda msg: batch_iterator.write(msg), global_step, writer, num_examples=3)


    model_filename = get_weights_file_path(config, f"{epoch:02d}")
    torch.save(
      {
      "epoch": epoch,
      "model_state_dict": model.state_dict(),
      "optimizer_state_dict" : optimizer.state_dict(),
      "global_step": global_step
    },
    model_filename
    )


## load tokenizer

In [ ]:
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00


In [ ]:
from evaluate import load

In [ ]:
config = configs[0]
tokenizer_src = get_or_build_tokenizer(config, dataset, config['lang_src'])
tokenizer_tgt = get_or_build_tokenizer(config, dataset, config['lang_tgt'])

path tokenizer :  /content/drive/MyDrive/Machine_translation/tokenizer_indonesian.json
tokenizer file ada
path tokenizer :  /content/drive/MyDrive/Machine_translation/tokenizer_english.json
tokenizer file ada


## build validation dataloader

In [ ]:
val_dataloader = get_ds(config, val_ds_raw, tokenizer_src, tokenizer_tgt, test=True)

apakah ada masalah di sos eos dan pad token?
max length of source sentence 56
max length of target sentence 61


In [ ]:
# FOLDER_PATH = '/content/drive/MyDrive/machine_translation'
# preloads = ["03", "03", "03"]
# configs =[ get_config(f"model_{i}", folder=FOLDER_PATH) for i in range(1, 4)]

## Train per model

In [ ]:
FOLDER_PATH = '/content/drive/MyDrive/Machine_translation'
# epoch = [10, 15, 20]
preload = ["24", "24", "24"]
# for ep in range(13, 31):
  # preload = [f"{ep-2}", f"{ep-2}", f"{ep-2}"]
  # print(preload)
configs =[ get_config(f"model_{i}", folder=FOLDER_PATH, preload=preload[i-1], epoch=35) for i in range(1, 4)]
for config in configs:
  subset = bagging(train_ds_raw, config['model_name'], folder=FOLDER_PATH)
  train_model(config, subset, val_dataloader, tokenizer_src, tokenizer_tgt)

Loading subset for model_1 from disk.
using device :  cuda
apakah ada masalah di sos eos dan pad token?
max length of source sentence 58
max length of target sentence 61
Preloading model /content/drive/MyDrive/Machine_translation/weights_model_1/tmodel_24.pt


Processing  epoch 25: 100%|██████████| 3934/3934 [05:00<00:00, 13.08it/s, Loss=2.625]


--------------------------------------------------------------------------------
Source: lalu semuanya berulang dan berulang lagi seperti itu, cara kerja yang sama yang diterapkan pada kompleks militer dan industri masa lalu.
Target: and it goes around and around and around, the same way that the military-industrial complex worked a long time ago.
Predicted: and then it is all over and over and over again , the same way , the same way that works in the same complex and industrial industry .
--------------------------------------------------------------------------------
Source: meskipun tikus digunakan di lab untuk mendukung kehidupan manusia, namun tikus juga dianggap hama.
Target: and even though they are used in labs to promote human lives, they are also considered pests.
Predicted: even though rats are used in the lab to support the human lives , but they also are considered of pests .
--------------------------------------------------------------------------------
Source: inilah t

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


BLEU: {'bleu': 0.1348131581635488, 'precisions': [0.5142857142857142, 0.208955223880597, 0.09375, 0.03278688524590164], 'brevity_penalty': 1.0, 'length_ratio': 1.1864406779661016, 'translation_length': 70, 'reference_length': 59}
Could not compute BLEU metric: 'int' object has no attribute 'get'


...

--------------------------------------------------------------------------------
Source: saya ellen, dan saya sangat terobsesi dengan pangan.
Target: i am ellen and i am totally obsessed with food.
Predicted: i was ellen , and i was obsessed with food .
--------------------------------------------------------------------------------
Source: ini akan menjadi lebih rumit daripada itu.
Target: it is going to be much more complicated than that.
Predicted: it will be more complicated than that .
--------------------------------------------------------------------------------
Source: ini adalah bulan ketika korea utara setuju untuk melucuti fasilitas nuklir mereka.
Target: now, this was a month when north korea agreed to dismantle its nuclear facilities.
Predicted: this is the moon when north korea ' s . inside of their nuclear facility .
BLEU: {'bleu': 0.23741074263374584, 'precisions': [0.6, 0.3125, 0.20689655172413793, 0.11538461538461539], 'brevity_penalty': 0.9178564384568926, 'length_r

Processing  epoch 33: 100%|██████████| 3934/3934 [05:01<00:00, 13.03it/s, Loss=2.552]


--------------------------------------------------------------------------------
Source: kita melakukan interogasi kecil-kecilan saat kita bertemu orang lain untuk membuat rangkuman riwayat hidup mereka di dalam pikiran kita.
Target: we do this little interrogation when we meet people to make a mental resume for them.
Predicted: we did a little interrogation we looked at the little while we met other people to run their gate , their lives in our minds .
--------------------------------------------------------------------------------
Source: tetapi, saya mengambil risiko setelah sadar bahwa kesempatan tak terduga sering kali datang setelah mengambil risiko, bukan tipe risiko berbahaya yang sudah saya teliti, tetapi risiko yang baik, risiko yang positif.
Target: but i took risks realizing that unforeseen opportunities often come from risk-taking not the hazardous, negative type that i studied, but the good ones, the positive risks.
Predicted: but i took a risk after realizing that unexpe

Processing  epoch 34: 100%|██████████| 3934/3934 [05:01<00:00, 13.06it/s, Loss=2.619]


--------------------------------------------------------------------------------
Source: saya berbicara tentang ini sebelumnya, dan seseorang yang brilian menghubungi saya dan berkata, "oke, apa yang dapat anda lakukan?"
Target: i spoke about this before, and a brilliant person got in touch with me and said, "okay, what can you do?"
Predicted: i talked about this before , and somebody who was brilliant to call me and said , " okay , what can you do ?"
--------------------------------------------------------------------------------
Source: mereka menolak dominasi kelompok atau individu. bahkan kepada atasan atau pakar.
Target: they do not let one group or one individual dominate, even if it is the boss, even if it is the expert.
Predicted: they refuse to be improved groups or an individual .
--------------------------------------------------------------------------------
Source: itu menurun manjadi 12 sampai 15 persen saja.
Target: that is down to about 12 to 15 percent.
Predicted: that

# Inference

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using device : ", device)
models = {}
preload = ["34", "34", "34"]
configs =[ get_config(f"model_{i}", folder=FOLDER_PATH, preload=preload[i-1]) for i in range(1, 4)]
for config in configs:
  model = get_model(config=config, vocab_src_len=tokenizer_src.get_vocab_size(), vocab_tgt_len=tokenizer_tgt.get_vocab_size()).to(device=device)
  model_filename = get_weights_file_path(config, config["preload"])
  state = torch.load(model_filename) # use GPU
  # state = torch.load(model_filename, map_location=torch.device('cpu')) # use cpu
  model.load_state_dict(state["model_state_dict"])
  model_name = config['model_name']
  models[model_name] = model

using device :  cuda


In [ ]:
val_dataloader = get_ds(config, val_ds_raw, tokenizer_src, tokenizer_tgt)

apakah ada masalah di sos eos dan pad token?
max length of source sentence 56
max length of target sentence 61


## Run validation

In [ ]:
run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config["seq_len"], device, lambda msg: print(msg), 0, None, num_examples=300)

--------------------------------------------------------------------------------
Source: tetapi bisakah lebih cepat? berapa nyawa yang bisa terselamatkan?
Target: but how much faster could we go, how many lives could we save?
Predicted: but can it be faster ? how many lives can be saved ?
--------------------------------------------------------------------------------
Source: itu seperti ada gempa haiti setiap delapan hari.
Target: that is a haiti earthquake every eight days.
Predicted: it was like there in haiti every eight days .
--------------------------------------------------------------------------------
Source: mengapa? ternyata pria memproduksi sperma yang sangat banyak.
Target: why? turns out that men make a lot of sperm.
Predicted: why ? well , it turns out that a guy produces a lot of sperm .
--------------------------------------------------------------------------------
Source: dan agar seimbang, saya percaya kita harus berada pada seluruh area tersebut tidak hanya melaku

{'preds': ['but can it be faster ? how many lives can be saved ?',
  'it was like there in haiti every eight days .',
  'why ? well , it turns out that a guy produces a lot of sperm .',
  'and to be balanced , i believe we should be in all of these areas are not just doing 50 years .',
  'and i have four quick observation i want to share with you today .',
  'a few years ago they had a protest .',
  'so the main point is , if we want to deliver many money to be good for good good good at this time , 100 years from now , a fairly rich dutch guy ?',
  'i started on the vagina of vagina because i worried about vaginas .',
  'so i ran in the cold , and i took photographs , and i took as many people i could , in february two years ago .',
  'earth is a place to forgive .',
  'and it will prevent 20 - use needle or 30 times .',
  'i will kill my father .',
  'that means that we want to be more healthy and everyday people .',
  "we decided to provide full of inspiration for all men but especi

## Majority vote

In [ ]:
import torch
from tqdm import tqdm

def majority_vote(models, validation_ds, tokenizer_src, tokenizer_tgt,
                  max_len, device, print_msg, global_state, writer,
                  num_examples=None):
    """
    models: dict name -> model
    validation_ds: iterable of batches (batch may have batch_size > 1)
    tokenizer_tgt: tokenizer target (harus punya token_to_id & decode/batch_decode)
    max_len: max decode length
    num_examples: jika bukan None -> batasi pemrosesan ke N contoh (contoh, bukan batch)
    """

    predictions = {"majority_vote": []}
    for model_name in models.keys():
        predictions[model_name] = []

    references = []

    sos_idx = tokenizer_tgt.token_to_id("[SOS]")
    eos_idx = tokenizer_tgt.token_to_id("[EOS]")

    processed = 0  # jumlah contoh yang sudah diproses

    with torch.no_grad():
        batch_iterator = tqdm(validation_ds, desc=f"Processing validation")
        for batch in batch_iterator:
            # stop kalau sudah mencapai batas contoh
            if num_examples is not None and processed >= num_examples:
                break

            encoder_input = batch["encoder_input"].to(device)
            encoder_mask = batch["encoder_mask"].to(device)
            batch_size = encoder_input.size(0)

            # Jika sisa quota contoh lebih sedikit dari batch_size, slice batch
            if num_examples is not None and processed + batch_size > num_examples:
                take = num_examples - processed  # >0
                encoder_input = encoder_input[:take]
                encoder_mask = encoder_mask[:take]
                src_texts = batch["src_text"][:take]
                tgt_texts = batch["tgt_text"][:take]
                batch_size = take
            else:
                src_texts = batch["src_text"]
                tgt_texts = batch["tgt_text"]

            # append references (per contoh)
            for t in tgt_texts:
                references.append(t)

            # untuk tiap model, lakukan decode batch
            for model_name, model in models.items():
                model.eval()
                # jalankan greedy decode batch (asumsi fungsi tersedia di scope)
                model_out = greedy_decode_batch(model, encoder_input, encoder_mask,
                                                tokenizer_src, tokenizer_tgt,
                                                max_len, device)
                # model_out: (batch_size, seq_len) tensor of token ids

                # ubah jadi list of token-id lists, lalu bersihin SOS/EOS
                ids_batch = model_out.detach().cpu().tolist()
                cleaned_batch = []
                for ids in ids_batch:
                    # drop leading SOS kalau ada
                    if len(ids) > 0 and ids[0] == sos_idx:
                        ids = ids[1:]
                    # crop at EOS (exclude EOS)
                    if eos_idx in ids:
                        ids = ids[:ids.index(eos_idx)]
                    cleaned_batch.append(ids)

                # decode to text
                if hasattr(tokenizer_tgt, "batch_decode"):
                    try:
                        pred_texts = tokenizer_tgt.batch_decode(cleaned_batch, skip_special_tokens=True)
                    except TypeError:
                        # some tokenizers expect different args
                        pred_texts = [tokenizer_tgt.decode(ids, skip_special_tokens=True) for ids in cleaned_batch]
                else:
                    pred_texts = [tokenizer_tgt.decode(ids, skip_special_tokens=True) for ids in cleaned_batch]

                # simpan prediksi per contoh
                for pt in pred_texts:
                    predictions[model_name].append(pt)

            processed += batch_size

            # optional: print progress message (jika print_msg dipakai)
            if print_msg:
                print_msg(f"Processed {processed} examples")

            # break if reached target
            if num_examples is not None and processed >= num_examples:
                break

    # Pastikan minimal contoh yang tersedia di semua model & references
    if len(models) == 0:
        return predictions, references

    n_preds_per_model = [len(predictions[m]) for m in models.keys()]
    n_examples = min(len(references), min(n_preds_per_model))

    if n_examples == 0:
        return predictions, references

    # lakukan majority voting per posisi kata untuk tiap contoh
    for idx in range(n_examples):
        list_of_predictions = []
        max_pred_len = 0
        for model_name in models.keys():
            pred_tokens = predictions[model_name][idx].split()
            list_of_predictions.append(pred_tokens)
            if len(pred_tokens) > max_pred_len:
                max_pred_len = len(pred_tokens)

        voted_words = []
        for pos in range(max_pred_len):
            word_count = {}
            for pred in list_of_predictions:
                if pos < len(pred):
                    w = pred[pos]
                    if w:
                        word_count[w] = word_count.get(w, 0) + 1
            if word_count:
                majority_word = max(word_count, key=word_count.get)
                voted_words.append(majority_word)
            else:
                break

        majority_sentence = " ".join(voted_words)
        predictions["majority_vote"].append(majority_sentence)

    return predictions, references


In [ ]:
def majority_vote(models, validation_ds, tokenizer_src, tokenizer_tgt,
                  max_len, device, print_msg, global_state, writer,
                  num_examples=None):
    """
    models: dict name -> model
    validation_ds: iterable of batches (expects batch size 1)
    tokenizer_tgt: tokeni<zer target
    max_len: max decode length (dipass dari caller)
    num_examples: jika bukan None -> batasi pemrosesan ke N contoh
    """

    import torch

    predictions = {"majority_vote": []}
    for model_name in models.keys():
        predictions[model_name] = []

    references = []

    with torch.no_grad():
        batch_iterator = tqdm(validation_ds, desc=f"Processing validation")
        for i, batch in enumerate(batch_iterator):
            if num_examples is not None and i >= num_examples:
                break

            # asumsi batch size 1
            encoder_input = batch["encoder_input"].to(device)
            encoder_mask = batch["encoder_mask"].to(device)
            assert encoder_input.size(0) == 1, "Batch size must be 1 for validation"

            target_text = batch["tgt_text"][0]
            references.append(target_text)

            for model_name, model in models.items():
                model.eval()
                model_out = greedy_decode(model, encoder_input, encoder_mask,
                                          tokenizer_src, tokenizer_tgt,
                                          max_len, device)
                # Pastikan model_out jadi list of ids sebelum decode
                try:
                    if isinstance(model_out, torch.Tensor):
                        token_ids = model_out.squeeze().detach().cpu().tolist()
                    else:
                        token_ids = list(model_out)  # fallback
                except Exception:
                    # fallback aman: coba convert via numpy
                    token_ids = model_out.detach().cpu().numpy().squeeze().tolist()

                # decode jadi string -- skip special tokens agar bersih
                pred_text = tokenizer_tgt.decode(token_ids, skip_special_tokens=True)
                # pred_text = detokenize_whitespace(pred_text)
                predictions[model_name].append(pred_text)

    # Tentukan berapa banyak contoh yang valid (semua model punya prediksi)
    if len(models) == 0:
        return predictions, references

    # pastikan kita hanya iterasi sampai jumlah prediksi minimum antar model & references
    n_preds_per_model = [len(predictions[m]) for m in models.keys()]
    n_examples = min(len(references), min(n_preds_per_model))

    if n_examples == 0:
        return predictions, references

    # lakukan majority voting per posisi kata untuk tiap contoh
    for idx in range(n_examples):
        list_of_predictions = []
        max_pred_len = 0
        for model_name in models.keys():
            # tokenisasi split pada spasi (sederhana); kalau tokenisasi berbeda, sesuaikan
            pred_tokens = predictions[model_name][idx].split()
            list_of_predictions.append(pred_tokens)
            if len(pred_tokens) > max_pred_len:
                max_pred_len = len(pred_tokens)

        # bangun kata hasil majority untuk posisi 0..max_pred_len-1
        voted_words = []
        for pos in range(max_pred_len):
            word_count = {}
            for pred in list_of_predictions:
                if pos < len(pred):
                    w = pred[pos]
                    if w:
                        word_count[w] = word_count.get(w, 0) + 1
            if word_count:
                # pilih kata dengan hit terbesar; tie -> pilih yang muncul pertama oleh max()
                majority_word = max(word_count, key=word_count.get)
                voted_words.append(majority_word)
            else:
                # tidak ada kata baki pada posisi ini (semua model pendek) -> stop
                break

        majority_sentence = " ".join(voted_words)
        predictions["majority_vote"].append(majority_sentence)

    return predictions, references


In [ ]:
preds, refs = majority_vote(models, val_dataloader, tokenizer_src, tokenizer_tgt, config["seq_len"], device, lambda msg: print(msg), 0, None)

Processing validation:   0%|          | 1/984 [00:00<11:16,  1.45it/s]

Processed 32 examples


Processing validation:   0%|          | 2/984 [00:01<10:57,  1.49it/s]

Processed 64 examples


Processing validation:   0%|          | 3/984 [00:01<10:03,  1.62it/s]

...

Processed 31424 examples


Processing validation: 100%|█████████▉| 983/984 [09:54<00:00,  1.76it/s]

Processed 31456 examples


Processing validation: 100%|██████████| 984/984 [09:54<00:00,  1.65it/s]

Processed 31471 examples


## Bleu Score

In [ ]:
def postprocessing(text):
  text = text.replace(" '", "'")
  text = text.replace(" ,", ",")
  text = text.replace(" ?", "?")
  text = text.replace(" .", ".")
  text = text.replace(" !", "!")
  return text


In [ ]:
bleu_metric = load("bleu")
for model, v in preds.items():
  preds_cleaned = [postprocessing(pred) for pred in v]
  print(f"BLEU scores for {model}:")
  for n in range(1, 5): # Loop through n-grams from 1 to 4
    result = bleu_metric.compute(predictions=preds_cleaned, references=refs, max_ngram=n)
    # Extract and print only the BLEU score for the current n-gram
    print(f"  {n}-gram BLEU: {result.get('bleu', 'N/A')}")

NameError: name 'postprocessing' is not defined

In [ ]:
bleu_metric = load("bleu")
for model, v in preds.items():
  # preds_cleaned = [postprocessing(pred) for pred in v]
  result = bleu_metric.compute(predictions=v, references=refs)
  print(f"BLEU score for {model} : ", result)

BLEU score for majority_vote :  {'bleu': 0.20646577988732337, 'precisions': [0.5261613556494353, 0.2659106052383929, 0.14947653531185498, 0.08688897915026401], 'brevity_penalty': 1.0, 'length_ratio': 1.076139631110176, 'translation_length': 613787, 'reference_length': 570360}
BLEU score for model_1 :  {'bleu': 0.22313249914463085, 'precisions': [0.5643404584258777, 0.2877850150573993, 0.16531563247858105, 0.09838262239498605], 'brevity_penalty': 0.9842431694547438, 'length_ratio': 0.9843660144470159, 'translation_length': 561443, 'reference_length': 570360}
BLEU score for model_2 :  {'bleu': 0.22317363097469922, 'precisions': [0.563926319737359, 0.2877662759215397, 0.16515395622979856, 0.09781450519415351], 'brevity_penalty': 0.9862891444153414, 'length_ratio': 0.986382284872712, 'translation_length': 562593, 'reference_length': 570360}
BLEU score for model_3 :  {'bleu': 0.22330272866130063, 'precisions': [0.5622795533580522, 0.286496637949149, 0.16460602580436773, 0.09775911972557351]

In [ ]:
# Example using NLTK for Corpus BLEU
# Make sure you have nltk installed: !pip install nltk

import nltk
# nltk.download('punkt') # Uncomment and run this line if you get a resource error

from nltk.translate.bleu_score import corpus_bleu

# Assuming you have your predictions and references as lists of sentences
# predictions = ["This is the predicted sentence 1", "This is predicted sentence 2"]
# references = [["This is the reference sentence 1"], ["This is reference sentence 2"]] # References should be a list of lists

# The 'refs' and 'preds_cleaned' variables should already be available from your previous code
# You might need to ensure 'refs' is a list of lists, where each inner list contains one reference translation
# For example:
# refs_for_nltk = [[ref] for ref in refs]

# Calculate Corpus BLEU
# score = corpus_bleu(refs_for_nltk, preds_cleaned)

# print(f"Corpus BLEU score (NLTK): {score}")

# If you want to see scores for different n-grams with NLTK:
# score_1_gram = corpus_bleu(refs_for_nltk, preds_cleaned, weights=(1, 0, 0, 0))
# score_2_gram = corpus_bleu(refs_for_nltk, preds_cleaned, weights=(0.5, 0.5, 0, 0))
# score_3_gram = corpus_bleu(refs_for_nltk, preds_cleaned, weights=(1/3, 1/3, 1/3, 0))
# score_4_gram = corpus_bleu(refs_for_nltk, preds_cleaned, weights=(0.25, 0.25, 0.25, 0.25)) # This is the default

# print(f"Corpus BLEU (NLTK) 1-gram: {score_1_gram}")
# print(f"Corpus BLEU (NLTK) 2-gram: {score_2_gram}")
# print(f"Corpus BLEU (NLTK) 3-gram: {score_3_gram}")
# print(f"Corpus BLEU (NLTK) 4-gram: {score_4_gram}")

# Using the existing 'preds_cleaned' and 'refs' variables
# Note: The 'references' argument for corpus_bleu in nltk expects a list of lists of reference translations.
# If you only have one reference per prediction, you need to format it like this:
refs_for_nltk = [[ref] for ref in refs]

# Calculate and print Corpus BLEU for different n-grams using NLTK
print("Corpus BLEU scores (NLTK):")
for n in range(1, 5):
    weights = tuple([1/n] * n + [0] * (4-n))
    score = corpus_bleu(refs_for_nltk, preds_cleaned, weights=weights)
    print(f"  {n}-gram BLEU: {score}")